In [1]:
import os
import pandas as pd
import cv2
import numpy as np
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras import layers
from tensorflow.keras.callbacks import ModelCheckpoint
import tensorflow as tf
import matplotlib.pyplot as plt

def plot_hist(hist):
    plt.plot(hist.history["accuracy"])
    plt.plot(hist.history["val_accuracy"])
    plt.title("model accuracy")
    plt.ylabel("accuracy")
    plt.xlabel("epoch")
    plt.legend(["train", "validation"], loc="upper left")
    plt.show()
def build_model(num_classes):
    inputs = layers.Input(shape=(IMAGE_SIZE, IMAGE_SIZE, 3))
    # x = img_augmentation(inputs)
    model = EfficientNetB0(include_top=False, input_tensor=inputs, weights=weights_path)

    # Freeze the pretrained weights
    model.trainable = False

    # Rebuild top
    x = layers.GlobalAveragePooling2D(name="avg_pool")(model.output)
    x = layers.BatchNormalization()(x)

    top_dropout_rate = 0.2
    x = layers.Dropout(top_dropout_rate, name="top_dropout")(x)
    # outputs = layers.Dense(NUM_CLASSES, activation="softmax", name="pred")(x)
    outputs = layers.Dense(NUM_CLASSES, name="pred")(x)
    # Compile
    model = tf.keras.Model(inputs, outputs, name="EfficientNet")
    optimizer = tf.keras.optimizers.Adam(learning_rate=1e-2)
    model.compile(
        optimizer=optimizer, loss="mean_squared_error", metrics=[tf.keras.metrics.RootMeanSquaredError()]
    )
    return model
data_dir = '../input/petfinder-pawpularity-score/train'
test_dir = '../input/petfinder-pawpularity-score/test'
labels = pd.read_csv('../input/petfinder-pawpularity-score/train.csv', usecols = ['Pawpularity'])
sample_submission = pd.read_csv("../input/petfinder-pawpularity-score/sample_submission.csv")
weights_path = '../input/imagenet-weights/imagenet_weights.hdf5'

batchSize = 64
IMAGE_SIZE = 224
NUM_CLASSES = 1

data = []
test = []

for i in os.listdir(test_dir):
    # print(i)
    img = cv2.imread(os.path.join(test_dir, i))
    img = cv2.resize(img, (IMAGE_SIZE, IMAGE_SIZE))
    test.append(img)
test = np.array(test)

brojac = 0
for i in os.listdir(data_dir):
    img = cv2.imread(os.path.join(data_dir, i))
    img = cv2.resize(img, (IMAGE_SIZE, IMAGE_SIZE))
    data.append(img)
    brojac += 1

train = np.array(data)

y = []
for i in labels['Pawpularity']:
    y.append(i/100)
y = np.array(y)
model = build_model(1)
epochs = 25  # @param {type: "slider", min:8, max:80}
hist = model.fit(x=train, y=y, epochs=epochs, validation_split=0.1, verbose=1)
# plot_hist(hist)
# model.load_weights(modelPath)
rez = model.predict(np.array(test).reshape(len(test), IMAGE_SIZE, IMAGE_SIZE, 3))
for i in range(len(rez)):
    rez[i] *= 100
sample_submission.Pawpularity = rez
sample_submission.to_csv("submission.csv", index=False)
# print(rez)


User settings:

   KMP_AFFINITY=granularity=fine,verbose,compact,1,0
   KMP_BLOCKTIME=0
   KMP_SETTINGS=1
   KMP_WARNINGS=0

Effective settings:

   KMP_ABORT_DELAY=0
   KMP_ADAPTIVE_LOCK_PROPS='1,1024'
   KMP_ALIGN_ALLOC=64
   KMP_ALL_THREADPRIVATE=128
   KMP_ATOMIC_MODE=2
   KMP_BLOCKTIME=0
   KMP_CPUINFO_FILE: value is not defined
   KMP_DETERMINISTIC_REDUCTION=false
   KMP_DEVICE_THREAD_LIMIT=2147483647
   KMP_DISP_NUM_BUFFERS=7
   KMP_DUPLICATE_LIB_OK=false
   KMP_ENABLE_TASK_THROTTLING=true
   KMP_FORCE_REDUCTION: value is not defined
   KMP_FOREIGN_THREADS_THREADPRIVATE=true
   KMP_FORKJOIN_BARRIER='2,2'
   KMP_FORKJOIN_BARRIER_PATTERN='hyper,hyper'
   KMP_GTID_MODE=3
   KMP_HANDLE_SIGNALS=false
   KMP_HOT_TEAMS_MAX_LEVEL=1
   KMP_HOT_TEAMS_MODE=0
   KMP_INIT_AT_FORK=true
   KMP_LIBRARY=throughput
   KMP_LOCK_KIND=queuing
   KMP_MALLOC_POOL_INCR=1M
   KMP_NUM_LOCKS_IN_BLOCK=1
   KMP_PLAIN_BARRIER='2,2'
   KMP_PLAIN_BARRIER_PATTERN='hyper,hyper'
   KMP_REDUCTION_BARRIER='1,1'
  

Epoch 1/25
279/279 [==============================] - 197s 678ms/step - loss: 0.9677 - root_mean_squared_error: 0.9837 - val_loss: 0.1440 - val_root_mean_squared_error: 0.3795
Epoch 2/25
279/279 [==============================] - 186s 665ms/step - loss: 0.2525 - root_mean_squared_error: 0.5025 - val_loss: 0.1617 - val_root_mean_squared_error: 0.4021
Epoch 3/25
279/279 [==============================] - 186s 667ms/step - loss: 0.1762 - root_mean_squared_error: 0.4198 - val_loss: 0.1691 - val_root_mean_squared_error: 0.4112
Epoch 4/25
279/279 [==============================] - 188s 673ms/step - loss: 0.1211 - root_mean_squared_error: 0.3481 - val_loss: 0.0924 - val_root_mean_squared_error: 0.3040
Epoch 5/25
279/279 [==============================] - 186s 668ms/step - loss: 0.0868 - root_mean_squared_error: 0.2945 - val_loss: 0.0808 - val_root_mean_squared_error: 0.2842
Epoch 6/25
279/279 [==============================] - 186s 666ms/step - loss: 0.0728 - root_mean_squared_error: 0.2699 -